In [113]:
# requirements.txt içeriğini oluşturma
requirements = """
langchain-openai
langchain-pinecone
langchain-community
langchain-text-splitters
"""

# requirements.txt dosyasını kaydetme
with open('requirements.txt', 'w') as f:
    f.write(requirements)


In [114]:
pip install python-docx


In [115]:
!pip install unstructured


In [116]:
!pip install "unstructured[docx]"


In [117]:
!pip install -r requirements.txt


In [118]:
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

import os
import glob
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [119]:
!mkdir data


mkdir: cannot create directory ‘data’: File exists


In [120]:
m_loader = DirectoryLoader(
    path="data",
    glob="**/*.txt"
)
docs = m_loader.load()

In [121]:
docs[0]

Document(metadata={'source': 'data/ai_verileri.txt'}, page_content='Kendini bir bot değil belki “Deniz Eren” isimi ile kendini tanıtacaktır. Temel Cümleler; Merhaba “Alle Group”a hoşgeldiniz. Ben Deniz Eren, size nasıl yardımcı olabilirim? Sorunuz için teşekkür ederim/bizimle iletişime geçtiğiniz için teşekkür ederim, “Cevap” Proje yöneticinize bildirdim en kısa sürede size dönüş sağlayacağım. Anlayışınız için teşekkür ederim, size yardımcı olmak için elimden geleni yapacağım. Teşekkür ederim, Size yardımcı olabileceğim başka bir konu var mı? her hangi bir sorunuz olursa lütfen bize ulaşmaktan çekinmeyin. Alle Group ile iletişime geçtiğiniz için teşekkür ederiz. Mutluluk dolu bir gün diliyorum.\n\nŞirket Hakkında Bilgiler; Hakkımızda Giderek daha fazla küresel bir hale gelen dünyanın en önemli ihtiyaçlarından biri, farklı kültürlerin birbirleriyle iletişim kurması, değişik dillerden insanların ve kurumların daha fazla temas kurarak ortaklıklar kurmasıdır. Bu şartlarda farklı dil ve kül

In [122]:
import os

# Tanımlamaları yapın (örnek olarak)
PINECONE_API_KEY = "88c6a8f6-fc21-4b1d-bb91-3951f3fb16f8"
OPENAI_API_KEY = "sk-proj-EhlifDQnRaJU2OWVa8rnT3BlbkFJCPYZxGEBVyZa6luzOX46"
index_name = "allechatbot"

# Ortam değişkenlerini ayarlayın
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

# OpenAIEmbeddings sınıfını kullanarak modeli başlatın
from langchain_openai import OpenAIEmbeddings

m_embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"
    )


In [123]:
#split the document into chunks
m_splitter = RecursiveCharacterTextSplitter()
chunks = m_splitter.split_documents(docs)

chunks[0]

Document(metadata={'source': 'data/ai_verileri.txt'}, page_content='Kendini bir bot değil belki “Deniz Eren” isimi ile kendini tanıtacaktır. Temel Cümleler; Merhaba “Alle Group”a hoşgeldiniz. Ben Deniz Eren, size nasıl yardımcı olabilirim? Sorunuz için teşekkür ederim/bizimle iletişime geçtiğiniz için teşekkür ederim, “Cevap” Proje yöneticinize bildirdim en kısa sürede size dönüş sağlayacağım. Anlayışınız için teşekkür ederim, size yardımcı olmak için elimden geleni yapacağım. Teşekkür ederim, Size yardımcı olabileceğim başka bir konu var mı? her hangi bir sorunuz olursa lütfen bize ulaşmaktan çekinmeyin. Alle Group ile iletişime geçtiğiniz için teşekkür ederiz. Mutluluk dolu bir gün diliyorum.')

In [124]:
m_vectorstore = PineconeVectorStore.from_documents(chunks, m_embeddings, index_name="allechatbot")

# Searching Pinecone

In [125]:
from pinecone import Pinecone
import os

class PineconeClientSingleton:
    _instance = None

    @staticmethod
    def get_instance():
        if PineconeClientSingleton._instance is None:
            # Initialize Pinecone client
            api_key = os.getenv("PINECONE_API_KEY")
            PineconeClientSingleton._instance = Pinecone(api_key=api_key)
            return PineconeClientSingleton._instance

    @staticmethod
    def create_index(name, dimension, metric, spec):
        if name not in PineconeClientSingleton._instance.list_indexes().names():
            PineconeClientSingleton._instance.create_index(
                name=name,
                dimension=dimension,
                metric=metric,
                spec=spec
            )

    @staticmethod
    def get_index(name):
        if name not in PineconeClientSingleton._instance.list_indexes().names():
            False
        else:
            return PineconeClientSingleton._instance.Index(name)

    @staticmethod
    def delete_index(name):
        if name in PineconeClientSingleton._instance.list_indexes().names():
            PineconeClientSingleton._instance.delete_index(name)


# Create the Pinecone client
pinecone_client = PineconeClientSingleton.get_instance()
index = PineconeClientSingleton.get_index("allechatbot")

In [126]:
vectorstore_client = PineconeVectorStore(index=index, embedding=m_embeddings)

In [127]:
query = "Tercüme Hizmetlerinizden bahsedebilir misiniz?"
results = vectorstore_client.similarity_search(query)
results

[Document(id='31a665ac-fc97-4b5f-b62a-258eef8c4a57', metadata={'source': 'data/AI-Veriler.txt'}, page_content='Tercüme edilmiş belgeyi nereden apostilleyeceğim? Apostil ve tasdik şerhi için kaymakamlık veya valiliklere başvurmanız lazım. Siz aposil yapıyor musunuz? Biz belgelere kaymakamlık veya valiliklerden apostil veya tasdik şerhi alabilmesi için süreci takip edebiliriz. Merhaba iyi günler, türkçeden almancaya 7 sayfalık belge çevirtecektim ama, gönderdiğim belgelere fiyat ve süre söylemeniz mümkün mü? Tabi ki, gönderdiğiniz belgeleri hemen proje yöneticilerimize iletiyorum çok kısa bir süre içinde sizinle iletişime geçecekler. Nasıl sipariş verebilirim? Buradan veya mail üzerinden dosyalarınızı bizimle paylaşabilirsiniz, proje yöneticilerimiz dosyanızı inceleyip ve sizinle iletişime geçektir ve onayınızın ardından süreci başlatacaktır. Ayrıca ofislerimize fiziki olarak da başvurabilirsiniz veya dosyalarınız kargo/kurye iletebilirsiniz. Dosyalar proje yöneticilerimize ulaşınca sizi

In [128]:
from langchain_openai import ChatOpenAI
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.prompts import PromptTemplate

# Initialize the language model
m_llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0
)

retriever = vectorstore_client.as_retriever()
m_qa = RetrievalQA.from_chain_type(
    llm=m_llm,
    chain_type="stuff",
    retriever=retriever
)

In [129]:
# Define the prompt template
prompt_template = """
Given the context of the following documents:
{context}

Please answer the user's query succinctly in no more than 100 words.

Query: {query}
"""

prompt = PromptTemplate(
    input_variables=["context", "query"],
    template=prompt_template
)

input_data = {
    "context": """

    """,
    "query":
        "Tercüme Hizmetlerinizden bahsedebilir misiniz?"
}

formatted_prompt = prompt.format(**input_data)
result = m_qa.invoke({"query": formatted_prompt})
print(result)

{'query': "\nGiven the context of the following documents:\n\n\n    \n\nPlease answer the user's query succinctly in no more than 100 words.\n\nQuery: Tercüme Hizmetlerinizden bahsedebilir misiniz?\n", 'result': 'Alle Tercüme Bürosu, yazılı ve sözlü çeviri hizmetleri sunmaktadır. Yeminli çeviri, yetkili makamlarca onaylı tercümanlar tarafından yapılır ve resmi belgeler için gereklidir. Normal çeviri, resmi onay gerektirmeyen içerikler için deneyimli tercümanlarca yapılır. Yazılı çeviri, materyallerin bir dilden diğerine çevrilmesidir. Sözlü çeviri, farklı dillerdeki konuşmaların çevirisidir ve yeminli veya normal olabilir. Ayrıca, Document Asist hizmetiyle belgelerin resmi kurumlar nezdinde onay süreçlerini takip ediyoruz.'}


In [130]:
input_data = {
    "context": """

    """,
    "query":
        "Peki ucretleriniz ne sekildedir? Ucretleriniz hakkinda bilgi verebilir misiniz?"
}

formatted_prompt = prompt.format(**input_data)
result = m_qa.invoke({"query": formatted_prompt})
print(result)

{'query': "\nGiven the context of the following documents:\n\n\n    \n\nPlease answer the user's query succinctly in no more than 100 words.\n\nQuery: Peki ucretleriniz ne sekildedir? Ucretleriniz hakkinda bilgi verebilir misiniz?\n", 'result': 'Ücretlendirmelerimiz sayfa başına değil, karakter veya kelime üzerinden hesaplanmaktadır. Tercüme hizmetlerimiz 1000 karakter veya 180 kelime üzerinden fiyatlandırılır. Dosyalarınızı bizimle paylaşırsanız, fiyat ve süre konusunda detaylı bilgi sağlayabiliriz. Noter onaylı işlemler için noter ücreti ayrıca ödenir.'}


In [131]:
input_data = {
    "context": """

    """,
    "query":
        "Sirketinizin kac subesi var"
}

formatted_prompt = prompt.format(**input_data)
result = m_qa.invoke({"query": formatted_prompt})
print(result)

{'query': "\nGiven the context of the following documents:\n\n\n    \n\nPlease answer the user's query succinctly in no more than 100 words.\n\nQuery: Sirketinizin kac subesi var\n", 'result': 'Şirketin iki şubesi bulunmaktadır: Ankara ve İstanbul.'}


In [132]:
import os

# GitHub kullanıcı bilgilerini girin
username = "chatbotalle"  # GitHub kullanıcı adınız
email = "chatbotalle@gmail.com"  # GitHub e-posta adresiniz

# Git yapılandırması
os.system(f"git config --global user.name '{username}'")
os.system(f"git config --global user.email '{email}'")


0

In [134]:
!git clone https://github.com/chatbotalle/main.git


Cloning into 'main'...
fatal: could not read Username for 'https://github.com': No such device or address
